In [77]:
pip install requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [78]:
pip install torch

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: torch in c:\users\vian martinez\appdata\local\programs\python\python312\lib\site-packages (2.5.1)




[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [79]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [80]:
import requests
import json
import torch
from transformers import XLNetTokenizer, XLNetForSequenceClassification

In [81]:
# API URLs
GET_ONM_DATA_API_URL = "http://174.138.31.241:6178/online-media/compute/getArticle"
MANAGE_ONM_SENTIMENT_URL = "http://174.138.31.241:6178/online-media/compute/manageSentimentOnArticle"


In [82]:
# Load the trained sentiment analysis model and tokenizer
MODEL_DIR = "C:/Users/Vian Martinez/Downloads/xlnet_sentiment_model_v2-20241218T110833Z-001/xlnet_sentiment_model_v2"
TOKENIZER_DIR = "C:/Users/Vian Martinez/Downloads/xlnet_sentiment_tokenizer_v2-20241218T110834Z-001/xlnet_sentiment_tokenizer_v2"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [83]:
# Load the saved model and tokenizer
model = XLNetForSequenceClassification.from_pretrained(MODEL_DIR).to(device)
tokenizer = XLNetTokenizer.from_pretrained(TOKENIZER_DIR)

In [84]:
# Map label indices to sentiments
label_map = {0: "negative", 1: "neutral", 2: "positive"}

In [85]:
def get_user_input():
    print("Enter the following details to fetch articles from the API:")
    project_id = input("Project ID: ")
    clipping_id = input("Clipping ID: ")
    end_date = input("End Date (YYYY-MM-DD HH:MM:SS): ")
    start_date = input("Start Date (YYYY-MM-DD HH:MM:SS): ")
    page = input("Page (default 1): ") or "1"
    limit = input("Limit (default 10): ") or "10"
    validation_list = input("Validation List (e.g., unvalidated): ")

    # Construct the request body
    body = {
        "projectId": project_id,
        "clippingId": clipping_id,
        "endDate": end_date,
        "startDate": start_date,
        "page": int(page),
        "limit": int(limit),
        "validationList": [validation_list]
    }

    return body


In [86]:
def analyze_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {key: val.to(device) for key, val in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_label = torch.argmax(logits, dim=1).item()
    return label_map[predicted_label]

In [87]:
def update_sentiment(api_url, project_id, clipping_id, original_id, start_date, end_date, page, limit, sentiment):
    payload = {
        "projectId": project_id,
        "clippingId": clipping_id,
        "originalId": original_id,  
        "startDate": start_date,
        "endDate": end_date,
        "page": page,
        "limit": limit,
        "validationList": ["unvalidated"],
        "sentiment": sentiment
    }
    headers = {"Content-Type": "application/json"}
    try:
        response = requests.post(api_url, json=payload, headers=headers)
        if response.status_code == 200:
            print(f"Sentiment updated successfully for Original ID {original_id}.")
        else:
            print(f"Failed to update sentiment for Original ID {original_id}. Status Code: {response.status_code}")
            print(f"Error: {response.text}")
    except Exception as e:
        print(f"An error occurred while updating sentiment for Original ID {original_id}: {e}")


In [88]:
def fetch_articles(api_url, body):
    headers = {"Content-Type": "application/json"}
    print("\nSending request to the API...")
    print("Request Body:", json.dumps(body, indent=4))  # Debugging request body
    try:
        response = requests.post(api_url, json=body, headers=headers)
        if response.status_code == 200:
            articles = response.json()
            print("\nArticles fetched successfully!")
            
            for article in articles:
                print("\nAnalyzing sentiment for article:")
                print(f"Title: {article.get('title', 'No Title Available')}")
                print(f"Original Sentiment: {article.get('sentiment', 'neutral')}")

                # Analyze sentiment of the content
                new_sentiment = analyze_sentiment(article.get('content', ''))
                article['sentiment'] = new_sentiment

                print(f"Updated Sentiment: {new_sentiment}")

                # Fetch the `originalId` for updating sentiment
                original_id = article.get('original_id')
                if not original_id:
                    print("\nError: 'originalId' not found in the article data. Skipping...")
                    continue

                # Update sentiment using the manageSentimentOnArticle API
                update_sentiment(
                    MANAGE_ONM_SENTIMENT_URL,
                    body["projectId"],
                    body["clippingId"],
                    original_id,  
                    body["startDate"],
                    body["endDate"],
                    body["page"],
                    body["limit"],
                    new_sentiment
                )
            
            print("\nAll articles processed successfully.")
        else:
            print(f"\nFailed to fetch articles. Status Code: {response.status_code}")
            print(f"Error: {response.text}")
    except Exception as e:
        print(f"\nAn error occurred: {e}")


In [89]:
if __name__ == "__main__":
    request_body = get_user_input()
    fetch_articles(GET_ONM_DATA_API_URL, request_body)

Enter the following details to fetch articles from the API:

Sending request to the API...
Request Body: {
    "projectId": "c960b616-7847-472f-9e16-e47941536fd7",
    "clippingId": "ffdc1ddf-c1d7-48d0-9875-d8f02bf0b291",
    "endDate": "2024-12-15 23:59:59",
    "startDate": "2024-12-15 00:00:00",
    "page": 1,
    "limit": 10,
    "validationList": [
        "unvalidated"
    ]
}

Articles fetched successfully!

Analyzing sentiment for article:
Title: ASDP Ambon Operasikan Lima Kapal di Libur Nataru
Original Sentiment: neutral
Updated Sentiment: neutral
Sentiment updated successfully for Original ID d9139cad5d64712056d135fd3bd2f7cfe44495ebbce0b106e46993cf3826c52c.

Analyzing sentiment for article:
Title: Pemprov Kepri Gelar Ujian PPPK di Batam, Penumpang Kapal RoRo Meningkat
Original Sentiment: neutral
Updated Sentiment: neutral
Sentiment updated successfully for Original ID c37d68d3bc2b4a925f2008f2c9930d581df0f61b3090d04fba2a1c8342f2f00f.

Analyzing sentiment for article:
Title: Je